In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.path import Path
import pandas as pd
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR
import copy
import time 
# Set random seed for reproducibility
torch.manual_seed(0)
np.random.seed(0)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Set default tensor type to float32
torch.set_default_tensor_type(torch.FloatTensor)

# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hidden1 = nn.Linear(2, 32)
        self.hidden2 = nn.Linear(32, 32)
        self.hidden3 = nn.Linear(32, 32)
        self.output = nn.Linear(32, 2)
    def forward(self, x):
        x = torch.tanh(self.hidden1(x))
        x = torch.tanh(self.hidden2(x))
        x = torch.tanh(self.hidden3(x))
        return self.output(x)  # return [u, v]
    # Define the problem domain using the given vertices
vertices = np.array([
    [0, 0],
    [0, 5],
    [5, 5],
    [8, 2],
    [10, 2],
    [10,0],
    [0,0]  # Closing the polygon
],dtype=np.float32)
path = Path(vertices)
def in_domain(x, y):
    points = np.column_stack((x.cpu().numpy(), y.cpu().numpy()))
    return torch.tensor(path.contains_points(points), dtype=torch.bool, device=device)

# Define boundary conditions
def BC_bottom(x, y):
    return ((y == 0) & (x >= 0) & (x <= 10)).squeeze()

def BC_left(x, y):
    return ((x == 0) & (y >= 0) & (y <= 5)).squeeze()

def BC_top(x, y):
    tol = 1e-6
    return (((y == 5) & (x >= 0) & (x <= 5)) | 
            (abs((y - 10) + x) < tol) & (x > 5) & (x < 8) |
            (y == 2) & (x >= 8) & (x <= 10)).squeeze()

def BC_right(x, y):
    return ((x == 10) & (y >= 0) & (y <= 2)).squeeze()

def BC_Slope(x, y):
    tol = 1e-6
    return (abs((y - 10) + x) < tol) & (x > 5) & (x < 8).squeeze()


class HuberLoss:
    def __init__(self, delta=0.5):    #<------------------------ Delta Huber loss
        self.delta = delta
        
    def __call__(self, y_true, y_pred):
        error = torch.abs(y_true - y_pred)
        quadratic = torch.min(error, torch.tensor(self.delta))
        linear = error - quadratic
        return torch.mean(0.5 * quadratic ** 2 + self.delta * linear)

class MultiViewPINN:
    def __init__(self):
        # Keep track of learned knowledge from different views
        self.view_knowledge = {}
        self.huber = HuberLoss(delta=0.5)   #<------------------------ Delta Huber loss
        
    def transfer_knowledge(self, source_net, target_net, layer_mapping):
        """Transfer weights from source network to target network"""
        for source_layer, target_layer in layer_mapping:
            target_net.state_dict()[target_layer].data.copy_(
                source_net.state_dict()[source_layer].data
            )

def BC(xy, net):
    x, y = xy[:, 0].unsqueeze(1), xy[:, 1].unsqueeze(1)
    
    output = net(xy)
    u = output[:, 0:1]  # แยก output channel แรกเป็น u
    v = output[:, 1:2]  # แยก output channel ที่สองเป็น v
    
    bc_b = BC_bottom(x, y)
    bc_l = BC_left(x, y)
    bc_t = BC_top(x, y)
    bc_r = BC_right(x, y)
    bc_slope = BC_Slope(x, y)
    
    huber = HuberLoss(delta=0.5)   #<------------------------ Delta Huber loss
    
    loss = huber(u[bc_b], torch.zeros_like(u[bc_b])) + huber(v[bc_b], torch.zeros_like(v[bc_b]))
    loss += huber(u[bc_l], torch.zeros_like(u[bc_l]))
    loss += huber(u[bc_r], torch.zeros_like(u[bc_r]))
    
    xy_top = xy[bc_t].requires_grad_(True)
    output_top = net(xy_top)
    u_top = output_top[:, 0:1]
    v_top = output_top[:, 1:2]
    
    u_x_top = torch.autograd.grad(u_top.sum(), xy_top, create_graph=True)[0][:, 0]
    u_y_top = torch.autograd.grad(u_top.sum(), xy_top, create_graph=True)[0][:, 1]
    v_x_top = torch.autograd.grad(v_top.sum(), xy_top, create_graph=True)[0][:, 0]
    v_y_top = torch.autograd.grad(v_top.sum(), xy_top, create_graph=True)[0][:, 1]
    
    E = 5.0  
    nu = 0.3 
    
    sigma_xx_top = (E / ((1 + nu) * (1 - 2 * nu))) * (u_x_top + nu * v_y_top)
    sigma_yy_top = (E / ((1 + nu) * (1 - 2 * nu))) * (v_y_top + nu * u_x_top)
    sigma_xy_top = E / (2 * (1 + nu)) * (u_y_top + v_x_top)
    
    loss += huber(sigma_xy_top, torch.zeros_like(sigma_xy_top))
    loss += huber(sigma_xx_top, torch.zeros_like(sigma_xx_top))
    loss += huber(sigma_yy_top, torch.zeros_like(sigma_yy_top))
    
    xy_right = xy[bc_r].requires_grad_(True)
    output_right = net(xy_right)
    u_right = output_right[:, 0:1]
    v_right = output_right[:, 1:2]
    
    u_y_right = torch.autograd.grad(u_right.sum(), xy_right, create_graph=True)[0][:, 1]
    v_x_right = torch.autograd.grad(v_right.sum(), xy_right, create_graph=True)[0][:, 0]
    
    sigma_xy_right = E / (2 * (1 + nu)) * (u_y_right + v_x_right)
    loss += huber(sigma_xy_right, torch.zeros_like(sigma_xy_right))

    xy_left = xy[bc_l].requires_grad_(True)
    output_left = net(xy_left)
    u_left = output_left[:, 0:1]
    v_left = output_left[:, 1:2]
    
    u_y_left = torch.autograd.grad(u_left.sum(), xy_left, create_graph=True)[0][:, 1]
    v_x_left = torch.autograd.grad(v_left.sum(), xy_left, create_graph=True)[0][:, 0]
    
    sigma_xy_left = E / (2 * (1 + nu)) * (u_y_left + v_x_left)
    loss += huber(sigma_xy_left, torch.zeros_like(sigma_xy_left))

    return loss


def load_fem_data(filename='FEM1_data.csv'):
    df = pd.read_csv(filename)
    x = torch.tensor(df['X'].values, dtype=torch.float32).unsqueeze(1).to(device)
    y = torch.tensor(df['Y'].values, dtype=torch.float32).unsqueeze(1).to(device)
    return x, y


def generate_training_data(fem_data_file, n_boundary=2000):
    x, y = load_fem_data(fem_data_file)
    
    # Keep only points inside the domain
    mask = in_domain(x, y)
    x, y = x[mask], y[mask]
    
    # Generate boundary points
    t = torch.linspace(0, 1, n_boundary, device=device).unsqueeze(1)
    
    # Define the boundary segments
    segments = [
        ([0, 0, 0], [0, 5, 5]),  # Left boundary
        ([0, 5, 8, 10, 10], [5, 5, 2, 2, 0]),  # Top and right boundary
        ([10, 0], [0, 0])  # Bottom boundary
    ]
    
    x_b = []
    y_b = []
    
    for segment in segments:
        x_seg = torch.tensor(np.interp(t.cpu().numpy(), np.linspace(0, 1, len(segment[0])), segment[0]), dtype=torch.float32, device=device)
        y_seg = torch.tensor(np.interp(t.cpu().numpy(), np.linspace(0, 1, len(segment[1])), segment[1]), dtype=torch.float32, device=device)
        x_b.append(x_seg)
        y_b.append(y_seg)
    
    x_b = torch.cat(x_b)
    y_b = torch.cat(y_b)
    
    return x, y, x_b, y_b


def PDE(x, y, net):
    xy = torch.cat([x, y], dim=1)
    xy.requires_grad = True
    
    output = net(xy)
    u = output[:, 0:1]
    v = output[:, 1:2]
    
    u_x = torch.autograd.grad(u.sum(), xy, create_graph=True)[0][:, 0].unsqueeze(1)
    u_y = torch.autograd.grad(u.sum(), xy, create_graph=True)[0][:, 1].unsqueeze(1)
    v_x = torch.autograd.grad(v.sum(), xy, create_graph=True)[0][:, 0].unsqueeze(1)
    v_y = torch.autograd.grad(v.sum(), xy, create_graph=True)[0][:, 1].unsqueeze(1)
    
    E = 5
    nu = 0.3
    gamma = 1.8
    
    sigma_xx = E / (1 - nu**2) * (u_x + nu * v_y)
    sigma_yy = E / (1 - nu**2) * (v_y + nu * u_x)
    sigma_xy = E / (2 * (1 + nu)) * (u_y + v_x)
    
    f_x = torch.zeros_like(x)
    f_y = -gamma * torch.ones_like(y)
    
    R_x = torch.autograd.grad(sigma_xx.sum(), xy, create_graph=True)[0][:, 0].unsqueeze(1) + \
          torch.autograd.grad(sigma_xy.sum(), xy, create_graph=True)[0][:, 1].unsqueeze(1) + f_x
    R_y = torch.autograd.grad(sigma_xy.sum(), xy, create_graph=True)[0][:, 0].unsqueeze(1) + \
          torch.autograd.grad(sigma_yy.sum(), xy, create_graph=True)[0][:, 1].unsqueeze(1) + f_y
    
    huber = HuberLoss(delta=0.5)  #<------------------------ Delta Huber loss
    loss_x = huber(R_x, torch.zeros_like(R_x))
    loss_y = huber(R_y, torch.zeros_like(R_y))
    
    return loss_x, loss_y

def calculate_batch_flops(batch_size):
    # Network parameters
    input_size = 2  # x, y coordinates
    hidden_size = 32 
    num_layers = 3
    output_size = 2  # u, v displacements
    
    # Forward pass operations
    linear_ops = batch_size * (
        (input_size * hidden_size) +  
        (hidden_size * hidden_size * (num_layers-1)) +  
        (hidden_size * output_size)  
    )
    activation_ops = batch_size * hidden_size * num_layers
    
    # PDE calculations
    derivative_ops = batch_size * (
        4 * (input_size * hidden_size + hidden_size * hidden_size * (num_layers-1) + hidden_size * output_size) +
        12 * hidden_size * num_layers
    )
    
    stress_ops = batch_size * (3 * 4)
    bc_ops = batch_size * (
        2 * (input_size * hidden_size + hidden_size * hidden_size * (num_layers-1) + hidden_size * output_size) +
        6 * hidden_size * num_layers
    )
    
    total_ops = linear_ops + activation_ops + derivative_ops + stress_ops + bc_ops
    return total_ops

def calculate_tflops(start_time, batch_flops):
    """คำนวณ TFLOPS โดยใช้เวลาที่ผ่านไปและจำนวน FLOPs ในแต่ละ batch"""
    elapsed = time.perf_counter() - start_time
    if elapsed <= 0:  # ป้องกันการหารด้วย 0
        return 0
    tflops = (batch_flops / elapsed) / 1e12
    return tflops  # คืนค่าจริงโดยไม่มีการ clamp ขั้นต่ำ


def train(net, optimizer, n_epochs, fem_data_file):
    def calculate_batch_flops(batch_size):
        # Network parameters
        input_size = 2  # x, y coordinates
        hidden_size = 32 
        num_layers = 3
        output_size = 2  # u, v displacements
        
        # Forward pass operations
        linear_ops = batch_size * (
            (input_size * hidden_size) +  # First layer
            (hidden_size * hidden_size * (num_layers-1)) +  # Hidden layers
            (hidden_size * output_size)  # Output layer
        )
        activation_ops = batch_size * hidden_size * num_layers  # tanh operations
        
        # PDE calculations (each derivative requires a backward pass)
        derivative_ops = batch_size * (
            4 * (input_size * hidden_size + hidden_size * hidden_size * (num_layers-1) + hidden_size * output_size) + # First derivatives (u_x, u_y, v_x, v_y)
            12 * hidden_size * num_layers  # Activation functions in backward pass
        )
        
        # Stress calculations
        stress_ops = batch_size * (
            3 * 4  # Basic arithmetic for each stress component
        )
        
        # BC operations
        bc_ops = batch_size * (
            2 * (input_size * hidden_size + hidden_size * hidden_size * (num_layers-1) + hidden_size * output_size) + # Forward pass for BC
            6 * hidden_size * num_layers  # Activation functions
        )
        
        total_ops = linear_ops + activation_ops + derivative_ops + stress_ops + bc_ops
        return total_ops

    start_time = time.time()
    start_flops = time.perf_counter()
    total_flops = 0

    multi_view = MultiViewPINN()
    huber = HuberLoss(delta=0.5) #<------------------------ Delta Huber loss
    
    scheduler = CosineAnnealingLR(optimizer, T_max=n_epochs, eta_min=1e-6)
# Removed: best_loss, patience, max_patience, prev_net variables
    best_loss = float('inf')
    patience = 0
    max_patience = 3000
    prev_net = None
    
    # สร้าง lists สำหรับเก็บข้อมูลระหว่างการเทรน
    log_data = {
        'epoch': [],
        'total_loss': [],
        'pde_x_loss': [],
        'pde_y_loss': [],
        'bc_loss': [],
        'learning_rate': [],
        'elapsed_time': [],
        'timestamp': [],
        'tflops': []
    }
    
    # สร้าง lists สำหรับเก็บข้อมูลสำหรับพล็อตกราฟ
    epoch_history = []
    loss_history = []
    pde_x_history = []
    pde_y_history = []
    bc_history = []
    
    for epoch in range(n_epochs):
        current_time = time.time()
        elapsed_time = current_time - start_time
        
        optimizer.zero_grad()
        
        x, y, x_b, y_b = generate_training_data(fem_data_file, 30000)
        batch_size = len(x) + len(x_b)  # Total points for this batch
        
        # Calculate FLOPs for this batch
        batch_flops = calculate_batch_flops(batch_size)
        total_flops += batch_flops
        
        xy = torch.cat([x, y], dim=1)
        xy_b = torch.cat([x_b, y_b], dim=1)
        
        loss_pde_x, loss_pde_y = PDE(x, y, net)
        loss_bc = BC(xy_b, net)
        loss = 2*loss_pde_x + 2*loss_pde_y + loss_bc
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=0.5)
        optimizer.step()
        scheduler.step()

        current_tflops = calculate_tflops(start_flops, batch_flops)
        
        # บันทึกข้อมูลทุก 100 epochs
        if (epoch + 1) % 100 == 0:
            log_data['epoch'].append(epoch)
            log_data['total_loss'].append(loss.item())
            log_data['pde_x_loss'].append(loss_pde_x.item())
            log_data['pde_y_loss'].append(loss_pde_y.item())
            log_data['bc_loss'].append(loss_bc.item())
            log_data['learning_rate'].append(scheduler.get_last_lr()[0])
            log_data['elapsed_time'].append(elapsed_time)
            log_data['timestamp'].append(time.strftime('%Y-%m-%d %H:%M:%S'))
            log_data['tflops'].append(current_tflops)
            
            # เก็บข้อมูลสำหรับพล็อตกราฟ
            epoch_history.append(epoch)
            loss_history.append(loss.item())
            pde_x_history.append(loss_pde_x.item())
            pde_y_history.append(loss_pde_y.item())
            bc_history.append(loss_bc.item())
            
            # แสดงผลและบันทึก CSV
            hours = int(elapsed_time // 3600)
            minutes = int((elapsed_time % 3600) // 60)
            seconds = int(elapsed_time % 60)
            
            print(f'Epoch {epoch+1}/{n_epochs} [{hours:02d}:{minutes:02d}:{seconds:02d}], '
                  f'Loss: {loss.item():.6f}, '
                  f'PDE_x: {loss_pde_x.item():.6f}, PDE_y: {loss_pde_y.item():.6f}, '
                  f'BC: {loss_bc.item():.6f}, LR: {scheduler.get_last_lr()[0]:.6f}, '
                  f'TFLOPs/s: {current_tflops:.6f}')
            
            # บันทึก CSV
            df_log = pd.DataFrame(log_data)
            df_log.to_csv('training_log.csv', index=False)

    # คำนวณข้อมูลสรุปเมื่อเสร็จสิ้นการเทรน
    end_time = time.time()
    training_time = end_time - start_time
    average_tflops = total_flops / (1e12 * training_time)
    
    hours = int(training_time // 3600)
    minutes = int((training_time % 3600) // 60)
    seconds = int(training_time % 60)
    
    # แสดงผลสรุป
    print(f"\nTraining completed in {hours:02d}:{minutes:02d}:{seconds:02d}")
    print(f"Average computational throughput: {average_tflops:.2f} TFLOPs/s")
    
    # บันทึกสรุปการ training
    with open('training_summary.txt', 'w') as f:
        f.write(f"Training Summary\n")
        f.write(f"===============\n")
        f.write(f"Total training time: {hours:02d}:{minutes:02d}:{seconds:02d}\n")
        f.write(f"Total epochs: {n_epochs}\n")
        f.write(f"Best loss: {best_loss:.6f}\n")
        f.write(f"Final loss: {loss.item():.6f}\n")
        f.write(f"Final PDE_x loss: {loss_pde_x.item():.6f}\n")
        f.write(f"Final PDE_y loss: {loss_pde_y.item():.6f}\n")
        f.write(f"Final BC loss: {loss_bc.item():.6f}\n")
        f.write(f"Final learning rate: {scheduler.get_last_lr()[0]:.6f}\n")
        f.write(f"Average computational throughput: {average_tflops:.5f} TFLOPs/s\n")
        f.write(f"Total FLOPs: {total_flops:,}\n")
    
    # พล็อตกราฟ Loss history
    plt.figure(figsize=(12, 8))
    
    # Plot total loss
    plt.subplot(2, 1, 1)
    plt.plot(epoch_history, loss_history, 'b-', label='Total Loss')
    plt.yscale('log')
    plt.xlabel('Epoch')
    plt.ylabel('Loss (log scale)')
    plt.title('Training Loss History')
    plt.legend()
    plt.grid(True)
    
    # Plot component losses
    plt.subplot(2, 1, 2)
    plt.plot(epoch_history, pde_x_history, 'r-', label='PDE_x Loss')
    plt.plot(epoch_history, pde_y_history, 'g-', label='PDE_y Loss')
    plt.plot(epoch_history, bc_history, 'b-', label='BC Loss')
    plt.yscale('log')
    plt.xlabel('Epoch')
    plt.ylabel('Component Losses (log scale)')
    plt.title('Component Losses History')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('loss_history.png')
    plt.close()
    
    # บันทึก CSV ครั้งสุดท้าย
    df_log = pd.DataFrame(log_data)
    df_log.to_csv('training_log.csv', index=False)
    
    return net, log_data
    
def plot_results(net, fem_data_file='FEM1_data.csv', output_filename='PiNN1_data.csv'):
    # Load FEM data
    df_fem = pd.read_csv(fem_data_file)
    X = df_fem['X'].values
    Y = df_fem['Y'].values
    
    # Create tensor from FEM data
    XY = torch.tensor(np.column_stack([X, Y]), dtype=torch.float32).to(device)
    
    # Compute displacements and stresses for all points
    XY.requires_grad_(True)
    with torch.enable_grad():
        output = net(XY)
        U = output[:, 0:1]  # แยก u
        V = output[:, 1:2]  # แยก v
        
        U_x = torch.autograd.grad(U.sum(), XY, create_graph=True)[0][:, 0]
        U_y = torch.autograd.grad(U.sum(), XY, create_graph=True)[0][:, 1]
        V_x = torch.autograd.grad(V.sum(), XY, create_graph=True)[0][:, 0]
        V_y = torch.autograd.grad(V.sum(), XY, create_graph=True)[0][:, 1]
        
        E = 5  # Young's modulus
        nu = 0.3  # Poisson's ratio
        
        sigma_xx = E / (1 - nu**2) * (U_x + nu * V_y)
        sigma_yy = E / (1 - nu**2) * (V_y + nu * U_x)
        sigma_xy = E / (2 * (1 + nu)) * (U_y + V_x)
    
    # Move tensors to CPU and convert to numpy
    U_full = U.detach().cpu().numpy().squeeze()/1000
    V_full = V.detach().cpu().numpy().squeeze()/1000
    sigma_xx_full = sigma_xx.detach().cpu().numpy().squeeze()*10
    sigma_yy_full = sigma_yy.detach().cpu().numpy().squeeze()*10
    sigma_xy_full = sigma_xy.detach().cpu().numpy().squeeze()*10
    
    # Calculate displacement magnitude
    magnitude = np.sqrt(U_full**2 + V_full**2)
    
    # Create DataFrame for CSV export
    df_out = pd.DataFrame({
        'X': X,
        'Y': Y,
        'ux': U_full,
        'uy': V_full,
        'sigma_xx': sigma_xx_full,
        'sigma_yy': sigma_yy_full,
        'sigma_xy': sigma_xy_full,
        'magnitude': magnitude
    })

    # Save the output to a CSV file
    df_out.to_csv(output_filename, index=False)
    print(f"Results saved to {output_filename}")
    
    # Apply domain mask for plotting
    mask = in_domain(XY[:, 0], XY[:, 1])
    mask_cpu = mask.cpu().numpy()
      
# Main execution
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    net = Net().to(device)  # สร้าง network เดียว
    optimizer = optim.Adam(net.parameters(), lr=0.001)  # ใช้ parameters จาก network เดียว
    
    fem_data_file = 'FEM1_data.csv'
    trained_net, _ = train(net=net, optimizer=optimizer, n_epochs=5000, fem_data_file=fem_data_file)

    try:
        plot_results(trained_net, fem_data_file=fem_data_file) 
    except Exception as e:
        print(f"Error in plot_results: {str(e)}")

Using device: cuda
Using device: cuda
Epoch 10/2000 [00:01:00], Loss: 1.706640, PDE_x: 0.004439, PDE_y: 0.716667, BC: 0.264428, LR: 0.001000
Epoch 20/2000 [00:01:41], Loss: 1.289925, PDE_x: 0.007161, PDE_y: 0.543887, BC: 0.187830, LR: 0.001000


KeyboardInterrupt: 